# migration digraph

In [1]:
""" 
# pour utiliser la version de `geo_nx` en dev au lieu de `pip install --editable .`
import os
import sys
new_path = os.getcwd()[:-26] + 'geo_nx'
sys.path.append(new_path)"""

import json, copy
from shapely import LineString, Point
import numpy as np
import geopandas as gpd
import pandas as pd
import geo_nx as gnx 
import networkx as nx 

from geo_nx import geom_to_crs, cast_id
from fonction_rtet import proximite, insertion_projection, association_stations
from interfaces import creation_pandas_stations
from afir import Afir

data_path = '../data/'
rtet_path = '../rtet/'
refnat = {'tiles': 'cartodbpositron', 'location': [46.3, 2.3], 'zoom_start': 7}
vl_central_2025 = (True, 400, 150, 60)
vl_central_2027 = (True, 600, 150, 60)
vl_global_2027 = (False, 300, 150, 60)
pl_global_2025 = (False, 1400, 350, 120)
pl_central_2027 = (True, 2800, 350, 120)


version = 'V1'
scenario = vl_global_2027

## Chargement RTET



In [2]:
central, pcum, pmax, dist = scenario
gr_rtet = gnx.from_geopandas_edgelist(gpd.read_file(rtet_path + version + '_rtet_edge.geojson'), 
                                 node_gdf=gpd.read_file(rtet_path + version + '_rtet_node.geojson'),
                                 node_attr=True, edge_attr=True)
#gr.clean_attributes()
if central:
    gr_core = nx.subgraph_view(gr_rtet, filter_edge=lambda x,y: gr_rtet[x][y]["core"]==True)
    gr =  gnx.GeoGraph(nx.induced_subgraph(gr_core, nx.utils.flatten(gr_core.edges())))
else:
    gr = gr_rtet

In [3]:
len(gr.nodes), len(gr.edges)

(2686, 2757)

In [4]:
dgr = gr.to_directed()

In [8]:
sub_auto = nx.subgraph_view(dgr, filter_edge=lambda x,y: dgr[x][y]["nature"] == 'troncon autoroute')
dgr_autoroute =  nx.induced_subgraph(sub_auto, nx.utils.flatten(sub_auto.edges()))

sub_hors_auto = nx.subgraph_view(dgr, filter_edge=lambda x,y: dgr[x][y]["nature"] == 'troncon hors autoroute')
dgr_hors_autoroute =  nx.induced_subgraph(sub_hors_auto, nx.utils.flatten(sub_hors_auto.edges()))

In [10]:
param_exp_ech = {'n_name': 'echangeurs', 'n_color': 'blue', 'n_marker_kwds': {'radius': 3, 'fill': True}}
param_exp_rp = {'n_name': 'rond-points', 'n_color': 'purple', 'n_marker_kwds': {'radius': 3, 'fill': True}}
param_exp_as = {'n_name': 'aire de service', 'n_color': 'red', 'n_marker_kwds': {'radius': 3, 'fill': True}}
param_exp_gr = {'e_popup': ['weight', 'source', 'target', "NATIONALRO", "ID", "nature", "core"], 
                'e_tooltip': ["source", "target", "nature", "core"], 'n_tooltip': ["node_id", "nature"], 'n_marker_kwds': {'radius': 2, 'fill': False}}

#carte = gr_echangeur.explore(refmap=refnat, edges=False, **param_exp_ech)
#carte = gr_rond_point.explore(refmap=carte, edges=False, **param_exp_rp)
#carte = gr_aire_service.explore(refmap=refnat, edges=False, **param_exp_as)
carte = dgr_autoroute.explore(refmap=refnat, n_color='black', e_name='edges autoroute', n_name='nodes autoroute', **param_exp_gr)
carte = dgr_hors_autoroute.explore(refmap=carte, e_color='green', e_name='edges hors autoroute', n_name='nodes hors autoroute', layercontrol=True, n_color='black', **param_exp_gr)
carte

AttributeError: 'DiGraph' object has no attribute 'explore'